In [ ]:
import sys
import numpy as np
import pandas as pd
import json
# from scipy.misc.pilutil import imread
from matplotlib.pyplot import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D
# from keras.layers.merge import Concatenate
from tensorflow.keras.layers import concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

# from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('drive/MyDrive/blue_team_1_web_attack.json','rb') as user_file:
  parsed_json = user_file.read()

In [ ]:
test_data = pd.DataFrame(json.loads(parsed_json))

In [ ]:
del parsed_json

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
test_data

,bgpDestinationAsNumber,bgpSourceAsNumber,biFlowEndMilliseconds,biFlowStartMilliseconds,destinationIPv4Address,destinationTransportPort,exercise_dst_ipv4_segment,extendedFlow,flowEndMilliseconds,flowStartMilliseconds,ingressInterface,ipClassOfService,ipVersion,octetDeltaCount,packetDeltaCount,protocolIdentifier,samplingAlgorithm,samplingInterval,sourceIPv4Address,sourceTransportPort,tcpControlBits,tcpWindowSize,timestamp,flowEndMilliseconds_Rev,flowStartMilliseconds_Rev,mplsLabelStackSection2,mplsLabelStackSection2_Rev,mplsLabelStackSection3,mplsLabelStackSection3_Rev,mplsLabelStackSection4,mplsLabelStackSection4_Rev,mplsTopLabelStackSection,mplsTopLabelStackSection_Rev,octetDeltaCount_Rev,packetDeltaCount_Rev,tcpControlBits_Rev,vlanId,applicationId,applicationName
0,0,45543,1553070773014,1553070773014,9.66.11.14,3306,blue-team-1,"{'tcp': '{""synSize"":44,""synTtl"":50}'}",1553070773014,1553070773014,1,0,4,44,1,6,0,0,27.3.0.196,59145,2,1024.0,1553070773014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,45543,1553070773014,1553070773014,9.66.11.14,110,blue-team-1,"{'tcp': '{""synSize"":44,""synTtl"":49}'}",1553070773014,1553070773014,1,0,4,44,1,6,0,0,27.3.0.196,59145,2,1024.0,1553070773014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,45543,1553070773014,1553070773014,9.66.11.14,1025,blue-team-1,"{'tcp': '{""synSize"":44,""synTtl"":47}'}",1553070773014,1553070773014,1,0,4,44,1,6,0,0,27.3.0.196,59145,2,1024.0,1553070773014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,45543,1553070775018,1553070775018,9.66.11.14,3306,blue-team-1,"{'tcp': '{""synSize"":44,""synTtl"":38}'}",1553070775018,1553070775018,1,0,4,44,1,6,0,0,27.3.0.196,59146,2,1024.0,1553070775018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,45543,1553070775018,1553070775018,9.66.11.14,110,blue-team-1,"{'tcp': '{""synSize"":44,""synTtl"":41}'}",1553070775018,1553070775018,1,0,4,44,1,6,0,0,27.3.0.196,59146,2,1024.0,1553070775018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,0,12389,1553089824375,1553089733034,9.66.11.14,80,blue-team-1,"{'http': '{""uaAppMin"":0,""uaOs"":68,""uaOsBld"":65...",1553089824372,1553089733034,1,0,4,1285,13,6,0,0,77.51.161.30,46532,26,29200.0,1553089824375,1.553090e+12,1.553090e+12,0x000000,0x000000,0x000000,0x000000,0x000000,0x000000,0x000000,0x000000,16199.0,6.0,26.0,0.0,50331728,HTTP
776,0,12389,1553089827749,1553089827749,9.66.11.14,80,blue-team-1,"{'http': '{""uaAppMin"":0,""uaOs"":68,""uaOsBld"":65...",1553089827749,1553089827749,1,0,4,642,1,6,0,0,77.51.161.30,46534,24,NaN,1553089827749,NaN,NaN,NaN,0x000000,NaN,0x000000,NaN,0x000000,NaN,0x000000,NaN,NaN,NaN,NaN,50331728,HTTP
777,0,12389,1553089828083,1553089828083,9.66.11.14,80,blue-team-1,"{'http': '{""uaAppMin"":0,""uaOs"":68,""uaOsBld"":65...",1553089828083,1553089828083,1,0,4,650,1,6,0,0,77.51.161.30,46532,24,NaN,1553089828083,NaN,NaN,NaN,0x000000,NaN,0x000000,NaN,0x000000,NaN,0x000000,NaN,NaN,NaN,NaN,50331728,HTTP
778,0,12389,1553089852674,1553089727525,9.66.11.14,80,blue-team-1,"{'http': '{""uaAppMin"":0,""uaOs"":68,""uaOsBld"":65...",1553089824372,1553089727525,1,0,4,1037,8,6,0,0,77.51.161.30,46480,26,29200.0,1553089852674,1.553090e+12,1.553090e+12,0x000000,0x000000,0x000000,0x000000,0x000000,0x000000,0x000000,0x000000,16340.0,9.0,30.0,0.0,50331728,HTTP


In [ ]:
test_data = test_data.drop(['bgpDestinationAsNumber','bgpSourceAsNumber','exercise_dst_ipv4_segment','extendedFlow','flowEndMilliseconds','flowStartMilliseconds','ingressInterface','ipClassOfService',
                'ipVersion','samplingInterval','tcpControlBits','tcpWindowSize','timestamp','flowEndMilliseconds_Rev','flowStartMilliseconds_Rev','mplsLabelStackSection2','mplsLabelStackSection2_Rev',
                'mplsLabelStackSection3',	'mplsLabelStackSection3_Rev',	'mplsLabelStackSection4',	'mplsLabelStackSection4_Rev',	'mplsTopLabelStackSection',	'mplsTopLabelStackSection_Rev','tcpControlBits_Rev','vlanId','applicationId','applicationName'], axis=1)

In [ ]:
test_data = test_data.drop(['samplingAlgorithm'],axis=1)

In [ ]:
df = test_data

In [ ]:
train_data = pd.read_csv('drive/MyDrive/output.csv')

In [ ]:
train_data

,ts,te,td,sa,da,sp,dp,pr,flg,fwd,stos,ipkt,ibyt,opkt,obyt,in,out,sas,das,smk,dmk,dtos,dir,nh,nhb,svln,dvln,ismc,odmc,idmc,osmc,mpls1,mpls2,mpls3,mpls4,mpls5,mpls6,mpls7,mpls8,mpls9,mpls10,cl,sl,al,ra,eng,exid,tr
0,2023-08-15 18:14:02,2023-08-15 18:14:03,1.090,10.0.0.2,10.128.0.2,37361,80.0,TCP,...AP.SF,0.0,0.0,10.0,854.0,7.0,7157.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
1,2023-08-15 18:14:02,2023-08-15 18:14:03,1.089,10.0.0.2,10.128.0.2,37362,80.0,TCP,...AP.SF,0.0,0.0,9.0,908.0,6.0,7105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
2,2023-08-15 18:14:02,2023-08-15 18:14:03,1.087,10.0.0.2,10.128.0.2,37363,80.0,TCP,...AP.SF,0.0,0.0,9.0,840.0,6.0,7105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
3,2023-08-15 18:14:02,2023-08-15 18:14:03,1.088,10.0.0.2,10.128.0.2,37364,80.0,TCP,...AP.SF,0.0,0.0,9.0,918.0,6.0,7105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
4,2023-08-15 18:14:02,2023-08-15 18:14:03,1.088,10.0.0.2,10.128.0.2,37365,80.0,TCP,...AP.SF,0.0,0.0,8.0,808.0,5.0,7053.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,2023-08-15 18:14:24,2023-08-15 18:14:24,0.064,10.0.0.2,10.128.0.2,37589,80.0,TCP,...AP.SF,0.0,0.0,6.0,502.0,4.0,3609.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
208,2023-08-15 18:14:24,2023-08-15 18:14:24,0.064,10.0.0.2,10.128.0.2,37590,80.0,TCP,...AP.SF,0.0,0.0,7.0,574.0,5.0,3661.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
209,Summary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,flows,bytes,packets,avg_bps,avg_pps,avg_bpp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_data=train_data.drop(train_data.index[-1])
train_data=train_data.drop(train_data.index[-1])
train_data=train_data.drop(train_data.index[-1])

In [ ]:
import datetime

datelist = []
for newdate in train_data['ts']:
  date = datetime.datetime.strptime(newdate, '%Y-%m-%d %H:%M:%S')
  timestamp = str((date - datetime.datetime(1970, 1, 1)).total_seconds()*1000)
  datelist.append(int(timestamp[:-2]))


In [ ]:
datelist2 = []
for newdate2 in train_data['te']:
  date = datetime.datetime.strptime(newdate2,'%Y-%m-%d %H:%M:%S')
  timestamp = str((date-datetime.datetime(1970,1,1)).total_seconds()*1000)
  datelist2.append(int(timestamp[:-2]))

In [ ]:
train_data['ts'] = datelist

In [ ]:
train_data['te'] = datelist2

In [ ]:
train_data

,ts,te,td,sa,da,sp,dp,pr,flg,fwd,stos,ipkt,ibyt,opkt,obyt,in,out,sas,das,smk,dmk,dtos,dir,nh,nhb,svln,dvln,ismc,odmc,idmc,osmc,mpls1,mpls2,mpls3,mpls4,mpls5,mpls6,mpls7,mpls8,mpls9,mpls10,cl,sl,al,ra,eng,exid,tr
0,1692123242000,1692123243000,1.090,10.0.0.2,10.128.0.2,37361,80.0,TCP,...AP.SF,0.0,0.0,10.0,854.0,7.0,7157.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
1,1692123242000,1692123243000,1.089,10.0.0.2,10.128.0.2,37362,80.0,TCP,...AP.SF,0.0,0.0,9.0,908.0,6.0,7105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
2,1692123242000,1692123243000,1.087,10.0.0.2,10.128.0.2,37363,80.0,TCP,...AP.SF,0.0,0.0,9.0,840.0,6.0,7105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
3,1692123242000,1692123243000,1.088,10.0.0.2,10.128.0.2,37364,80.0,TCP,...AP.SF,0.0,0.0,9.0,918.0,6.0,7105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
4,1692123242000,1692123243000,1.088,10.0.0.2,10.128.0.2,37365,80.0,TCP,...AP.SF,0.0,0.0,8.0,808.0,5.0,7053.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,1692123264000,1692123264000,0.064,10.0.0.2,10.128.0.2,37586,80.0,TCP,...AP.SF,0.0,0.0,8.0,630.0,6.0,3713.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
205,1692123264000,1692123264000,0.064,10.0.0.2,10.128.0.2,37587,80.0,TCP,...AP.SF,0.0,0.0,6.0,473.0,4.0,3609.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
206,1692123264000,1692123264000,0.064,10.0.0.2,10.128.0.2,37588,80.0,TCP,...AP.SF,0.0,0.0,6.0,561.0,4.0,3609.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299
207,1692123264000,1692123264000,0.064,10.0.0.2,10.128.0.2,37589,80.0,TCP,...AP.SF,0.0,0.0,6.0,502.0,4.0,3609.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0.0.0,0.0.0.0,0.0,0.0,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.0,0.0,0.0,127.0.0.1,0/0,1.0,2023-07-31 12:46:41.299


In [ ]:
train_data = train_data.drop(['td','flg','fwd','stos','in','out','sas','das','smk','dmk','dtos','dir','nh','nhb','svln','dvln','ismc','odmc','idmc','osmc','mpls1',
                              'mpls2','mpls3','mpls4','mpls5','mpls6','mpls7','mpls8','mpls9','mpls10','cl','sl','al','ra','eng','exid','tr'],axis=1)

In [ ]:
train_data

,ts,te,sa,da,sp,dp,pr,ipkt,ibyt,opkt,obyt
0,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37361,80.0,TCP,10.0,854.0,7.0,7157.0
1,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37362,80.0,TCP,9.0,908.0,6.0,7105.0
2,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37363,80.0,TCP,9.0,840.0,6.0,7105.0
3,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37364,80.0,TCP,9.0,918.0,6.0,7105.0
4,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37365,80.0,TCP,8.0,808.0,5.0,7053.0
...,...,...,...,...,...,...,...,...,...,...,...
204,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37586,80.0,TCP,8.0,630.0,6.0,3713.0
205,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37587,80.0,TCP,6.0,473.0,4.0,3609.0
206,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37588,80.0,TCP,6.0,561.0,4.0,3609.0
207,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37589,80.0,TCP,6.0,502.0,4.0,3609.0


In [ ]:
train_data.rename(columns = {'ts':'biFlowStartMilliseconds','te':'biFlowEndMilliseconds','sa':'sourceIPv4Address','da':'destinationIPv4Address','sp':'sourceTransportPort',
                             'dp':'destinationTransportPort','pr':'protocolIdentifier','ipkt':'packetDeltaCount','opkt':'packetDeltaCount_Rev','ibyt':'octetDeltaCount','obyt':'octetDeltaCount_Rev'}, inplace = True)

In [ ]:
train_data['protocolIdentifier'] = 6

In [ ]:
train_data

,biFlowStartMilliseconds,biFlowEndMilliseconds,sourceIPv4Address,destinationIPv4Address,sourceTransportPort,destinationTransportPort,protocolIdentifier,packetDeltaCount,octetDeltaCount,packetDeltaCount_Rev,octetDeltaCount_Rev
0,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37361,80.0,6,10.0,854.0,7.0,7157.0
1,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37362,80.0,6,9.0,908.0,6.0,7105.0
2,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37363,80.0,6,9.0,840.0,6.0,7105.0
3,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37364,80.0,6,9.0,918.0,6.0,7105.0
4,1692123242000,1692123243000,10.0.0.2,10.128.0.2,37365,80.0,6,8.0,808.0,5.0,7053.0
...,...,...,...,...,...,...,...,...,...,...,...
204,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37586,80.0,6,8.0,630.0,6.0,3713.0
205,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37587,80.0,6,6.0,473.0,4.0,3609.0
206,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37588,80.0,6,6.0,561.0,4.0,3609.0
207,1692123264000,1692123264000,10.0.0.2,10.128.0.2,37589,80.0,6,6.0,502.0,4.0,3609.0


In [ ]:
train_data_normal = test_data[-209:]
test_data = test_data.drop(test_data.index[-209:])

In [ ]:
val_train = train_data[-80:]
train_data = train_data.drop(train_data.index[-80:])
val_normal = train_data_normal[-80:]
train_data_normal = train_data_normal.drop(train_data_normal.index[-80:])

In [ ]:
test_data

,biFlowEndMilliseconds,biFlowStartMilliseconds,destinationIPv4Address,destinationTransportPort,octetDeltaCount,packetDeltaCount,protocolIdentifier,sourceIPv4Address,sourceTransportPort,octetDeltaCount_Rev,packetDeltaCount_Rev
0,1553070773014,1553070773014,9.66.11.14,3306,44,1,6,27.3.0.196,59145,NaN,NaN
1,1553070773014,1553070773014,9.66.11.14,110,44,1,6,27.3.0.196,59145,NaN,NaN
2,1553070773014,1553070773014,9.66.11.14,1025,44,1,6,27.3.0.196,59145,NaN,NaN
3,1553070775018,1553070775018,9.66.11.14,3306,44,1,6,27.3.0.196,59146,NaN,NaN
4,1553070775018,1553070775018,9.66.11.14,110,44,1,6,27.3.0.196,59146,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
566,1553070943203,1553070943203,9.66.11.14,9618,44,1,6,27.3.0.196,59146,NaN,NaN
567,1553070944203,1553070944203,9.66.11.14,631,44,1,6,27.3.0.196,59145,NaN,NaN
568,1553070944203,1553070944203,9.66.11.14,2323,44,1,6,27.3.0.196,59145,NaN,NaN
569,1553070944203,1553070944203,9.66.11.14,3077,44,1,6,27.3.0.196,59145,NaN,NaN


In [ ]:
test_data = test_data.fillna(0)
train_data = train_data.fillna(0)
train_data_normal = train_data_normal.fillna(0)
val_train = val_train.fillna(0)
val_normal = val_normal.fillna(0)

In [ ]:
import ipaddress

In [ ]:
ip_list = []
for addr in test_data['destinationIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list.append(addr)
test_data['destinationIPv4Address'] = ip_list

ip_list2 = []
for addr in test_data['sourceIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list2.append(addr)
test_data['sourceIPv4Address'] = ip_list2

In [ ]:
ip_list = []
for addr in train_data['destinationIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list.append(addr)
train_data['destinationIPv4Address'] = ip_list

ip_list2 = []
for addr in train_data['sourceIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list2.append(addr)
train_data['sourceIPv4Address'] = ip_list2

In [ ]:
ip_list = []
for addr in train_data_normal['destinationIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list.append(addr)
train_data_normal['destinationIPv4Address'] = ip_list

ip_list2 = []
for addr in train_data_normal['sourceIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list2.append(addr)
train_data_normal['sourceIPv4Address'] = ip_list2

In [ ]:
ip_list = []
for addr in val_train['destinationIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list.append(addr)
val_train['destinationIPv4Address'] = ip_list

ip_list2 = []
for addr in val_train['sourceIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list2.append(addr)
val_train['sourceIPv4Address'] = ip_list2

In [ ]:
ip_list = []
for addr in val_normal['destinationIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list.append(addr)
val_normal['destinationIPv4Address'] = ip_list

ip_list2 = []
for addr in val_normal['sourceIPv4Address']:
   addr =  int(ipaddress.IPv4Address(addr))
   ip_list2.append(addr)
val_normal['sourceIPv4Address'] = ip_list2

In [ ]:
#removing some more irrelevant features


test_data['flowDuration'] = test_data['biFlowEndMilliseconds']-test_data['biFlowStartMilliseconds']
test_data = test_data.drop(['biFlowEndMilliseconds','biFlowStartMilliseconds','destinationIPv4Address','sourceIPv4Address','sourceTransportPort','octetDeltaCount_Rev','packetDeltaCount_Rev'],axis=1)

In [ ]:
train_data['flowDuration'] = train_data['biFlowEndMilliseconds']-train_data['biFlowStartMilliseconds']
train_data = train_data.drop(['biFlowEndMilliseconds','biFlowStartMilliseconds','destinationIPv4Address','sourceIPv4Address','sourceTransportPort','octetDeltaCount_Rev','packetDeltaCount_Rev'],axis=1)

train_data_normal['flowDuration'] = train_data_normal['biFlowEndMilliseconds']-train_data_normal['biFlowStartMilliseconds']
train_data_normal = train_data_normal.drop(['biFlowEndMilliseconds','biFlowStartMilliseconds','destinationIPv4Address','sourceIPv4Address','sourceTransportPort','octetDeltaCount_Rev','packetDeltaCount_Rev'],axis=1)

val_train['flowDuration'] = val_train['biFlowEndMilliseconds']-val_train['biFlowStartMilliseconds']
val_train = val_train.drop(['biFlowEndMilliseconds','biFlowStartMilliseconds','destinationIPv4Address','sourceIPv4Address','sourceTransportPort','octetDeltaCount_Rev','packetDeltaCount_Rev'],axis=1)

val_normal['flowDuration'] = val_normal['biFlowEndMilliseconds']-val_normal['biFlowStartMilliseconds']
val_normal = val_normal.drop(['biFlowEndMilliseconds','biFlowStartMilliseconds','destinationIPv4Address','sourceIPv4Address','sourceTransportPort','octetDeltaCount_Rev','packetDeltaCount_Rev'],axis=1)



In [ ]:
train_data

,destinationTransportPort,protocolIdentifier,packetDeltaCount,octetDeltaCount,flowDuration
0,80.0,6,10.0,854.0,1000
1,80.0,6,9.0,908.0,1000
2,80.0,6,9.0,840.0,1000
3,80.0,6,9.0,918.0,1000
4,80.0,6,8.0,808.0,1000
...,...,...,...,...,...
124,80.0,6,7.0,545.0,0
125,80.0,6,8.0,645.0,0
126,80.0,6,6.0,477.0,0
127,80.0,6,7.0,554.0,0


In [ ]:
for i in test_data.columns:
  j=1
  val = max(test_data[i])
  # print(val)
  while val>1:
    j=j*10
    val = val/10
    # print(val)
  # print(j)
  test_data[i] = test_data[i]/j
  # print(max(test_data[i]))
  # print("########")

In [ ]:
for i in train_data.columns:
  train_data[i] = train_data[i].astype(int)

for i in train_data_normal.columns:
  train_data_normal[i] = train_data_normal[i].astype(int)

for i in val_train.columns:
  val_train[i] = val_train[i].astype(int)

for i in val_normal.columns:
  val_normal[i] = val_normal[i].astype(int)

In [ ]:
for i in train_data.columns:
  j=1
  val = max(train_data[i])
  # print(val)
  while val>1:
    j=j*10
    val = val/10
    # print(val)
  # print(j)
  train_data[i] = train_data[i]/j
  # print(max(test_data[i]))
  # print("########")

for i in train_data_normal.columns:
  j=1
  val = max(train_data_normal[i])
  # print(val)
  while val>1:
    j=j*10
    val = val/10
    # print(val)
  # print(j)
  train_data_normal[i] = train_data_normal[i]/j
  # print(max(test_data[i]))
  # print("########")

for i in val_train.columns:
  j=1
  val = max(val_train[i])
  # print(val)
  while val>1:
    j=j*10
    val = val/10
    # print(val)
  # print(j)
  val_train[i] = val_train[i]/j
  # print(max(test_data[i]))
  # print("########")

for i in val_normal.columns:
  j=1
  val = max(val_normal[i])
  # print(val)
  while val>1:
    j=j*10
    val = val/10
    # print(val)
  # print(j)
  val_normal[i] = val_normal[i]/j
  # print(max(test_data[i]))
  # print("########")

In [ ]:
#multiplying dest port by 10

test_data['destinationTransportPort'] = test_data['destinationTransportPort']*10
train_data['destinationTransportPort'] = train_data['destinationTransportPort']*10
train_data_normal['destinationTransportPort'] = train_data_normal['destinationTransportPort']*10
val_train['destinationTransportPort'] = val_train['destinationTransportPort']*10
val_normal['destinationTransportPort'] = val_normal['destinationTransportPort']*10

In [ ]:
def initialize_weights(shape, dtype=None):

    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [ ]:
def initialize_bias(shape, dtype=None):

    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [ ]:
def get_siamese_model(input_shape):


    left_input = Input(input_shape)
    right_input = Input(input_shape)


    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))


    model.add(Dense(128, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    model.add(Flatten())

    encoded_l = model(left_input)
    encoded_r = model(right_input)


    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])


    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)

    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)


    return siamese_net

In [ ]:
model = get_siamese_model((1,5))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1, 5)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1, 5)]       0           []                               
                                                                                                  
 sequential_1 (Sequential)      (None, 128)          8704        ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 128)          0           ['sequential_1[0][0]',     

In [ ]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if(s=="train"):
      X = train_data
      Y = train_data_normal
      n = len(train_data)
    else:
      X = val_train
      Y = val_normal
      n = len(val_train)

    categories = rng.choice(n,size=(batch_size,),replace=False)

    pairs=[np.zeros((batch_size,1,5)) for i in range(2)]


    targets=np.zeros((batch_size,))


    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        pairs[0][i,:,:] = np.array(X.iloc[category]).reshape(1,5)



        if i >= batch_size // 2:
            pairs[1][i,:,:] = np.array(X.iloc[category]).reshape(1,5)
        else:

            pairs[1][i,:,:] = np.array(Y.iloc[category]).reshape(1,5)

    return pairs, targets

In [ ]:
def generate(batch_size):

    while True:
        pairs, targets = get_batch(batch_size)
        yield (pairs, targets)

In [ ]:
def make_oneshot_task(N, s="val", language=None):

    if(s=="train"):
      X = train_data
      Y = train_data_normal
      n = len(train_data)
    else:
      X = val_train
      Y = val_normal
      n = len(val_train)


    Z = np.concatenate([X,Y])
    categories = rng.choice(range(2*n),size=(N,),replace=False)
    true_category = categories[0]

    test_image = np.asarray([Z[true_category,:]]*N).reshape(N,1,5)
    support_set = Z[categories,:]
    support_set = support_set.reshape(N,1,5)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

In [ ]:
pairs,targets = make_oneshot_task(10)

In [ ]:
def test_oneshot(model, N, k, s = "val", verbose = 0):

    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [ ]:
# Hyper parameters
evaluate_every = 20 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 600 # No. of training iterations
N_way = 30 # how many classes for testing one-shot tasks
n_val = 25 # how many one-shot tasks to validate on
best = -1

In [ ]:
model_path = '/content/'

In [ ]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss))
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------

 ------------- 

Time for 20 iterations: 0.01886681318283081 mins
Train Loss: 0.720152735710144
Evaluating model on 25 random 30 way one-shot learning tasks ... 

1/1 [==============================] - 0s 33ms/step
Got an average of 60.0% 30 way one-shot learning accuracy 

Current best: 60.0, previous best: -1

 ------------- 

Time for 40 iterations: 0.09699549277623494 mins
Train Loss: 0.7170601487159729
Evaluating model on 25 random 30 way one-shot learning tasks ... 

1/1 [==============================] - 0s 35ms/step
Got an average of 64.0% 30 way one-shot learning accuracy 

Current best: 64.0, previous best: 60.0

 ------------- 

Time for 60 iterations: 0.2008913000424703 mins
Train Loss: 0.71327805519104
Evaluating model on 25 random 30 way one-shot learning tasks ... 

1/1 [==============================] - 0s 24ms/step
Got an average of 76.0% 30 way one-shot learning accuracy 

Current best: 76.0, previous b

# Testing

In [ ]:
def test(n):
    N = len(test_data)
    test_image = np.asarray([train_data.iloc[n,:]]*N).reshape(N,1,5)
    support_set = np.asarray(test_data)
    support_set = support_set.reshape(N,1,5)
    pairs = [test_image,support_set]

    score = model.predict(pairs)

    return score

In [ ]:
# score = []
# for i in range(len(train_data)):
#   score_list = test(i)
#   score_val = []
#   for i in score_list:
#     score_val.append(i[0])
#   score.append(score_val)
score = test(0)

18/18 [==============================] - 0s 2ms/step


In [ ]:
score
score_list = []
for i in score:
  score_list.append(i[0])

In [ ]:
score

array([[0.20782168],
       [0.19381644],
       [0.19775055],
       [0.20782168],
       [0.19381644],
       [0.19775055],
       [0.19419645],
       [0.19524978],
       [0.19344127],
       [0.2179016 ],
       [0.19524978],
       [0.19419645],
       [0.19344127],
       [0.21379064],
       [0.19444013],
       [0.19571789],
       [0.21959662],
       [0.19571789],
       [0.19444013],
       [0.21959662],
       [0.19382073],
       [0.1935734 ],
       [0.23011254],
       [0.1935734 ],
       [0.23011254],
       [0.19382073],
       [0.19585976],
       [0.19392316],
       [0.19762062],
       [0.193437  ],
       [0.19585977],
       [0.19392316],
       [0.193437  ],
       [0.19762062],
       [0.19394022],
       [0.23404597],
       [0.1939573 ],
       [0.19394022],
       [0.1939573 ],
       [0.23404597],
       [0.20077924],
       [0.19761193],
       [0.20079242],
       [0.20079242],
       [0.20077924],
       [0.19761193],
       [0.19382927],
       [0.219

In [ ]:
max_val = 0
iter=0
ind=0
for i in score_list:
  if(i>max_val):
    max_val = i
    ind=iter
    print(i)
    print(ind)
  iter+=1

0.20782168
0
0.2179016
9
0.21959662
16
0.23011254
22
0.23404597
35
0.24267186
54
0.2759742
62
0.36884075
65
0.47418532
72
0.4741918
132
0.48292065
157
0.5058921
178
0.50771827
300
0.5264397
339
0.5544342
484


In [ ]:
min_val = 1
iter=0
ind=0
for i in score_list:
  if(i<max_val):
    min_val = i
    ind=iter
    print(i)
    print(ind)
  iter+=1

0.20782168
0
0.19381644
1
0.19775055
2
0.20782168
3
0.19381644
4
0.19775055
5
0.19419645
6
0.19524978
7
0.19344127
8
0.2179016
9
0.19524978
10
0.19419645
11
0.19344127
12
0.21379064
13
0.19444013
14
0.19571789
15
0.21959662
16
0.19571789
17
0.19444013
18
0.21959662
19
0.19382073
20
0.1935734
21
0.23011254
22
0.1935734
23
0.23011254
24
0.19382073
25
0.19585976
26
0.19392316
27
0.19762062
28
0.193437
29
0.19585977
30
0.19392316
31
0.193437
32
0.19762062
33
0.19394022
34
0.23404597
35
0.1939573
36
0.19394022
37
0.1939573
38
0.23404597
39
0.20077924
40
0.19761193
41
0.20079242
42
0.20079242
43
0.20077924
44
0.19761193
45
0.19382927
46
0.21973112
47
0.20819524
48
0.19345407
49
0.19382927
50
0.21973112
51
0.20819524
52
0.19345407
53
0.24267186
54
0.19344555
55
0.19850118
56
0.19344555
57
0.19850118
58
0.24267186
59
0.19956793
60
0.21973582
61
0.2759742
62
0.21507801
63
0.19720952
64
0.36884075
65
0.25051194
66
0.36884075
67
0.25051194
68
0.21507801
69
0.19720952
70
0.22700578
71
0.47418532
7

In [ ]:
ind

570

In [ ]:
df.iloc[570]

biFlowEndMilliseconds       1553070944203
biFlowStartMilliseconds     1553070944203
destinationIPv4Address         9.66.11.14
destinationTransportPort               89
octetDeltaCount                        44
packetDeltaCount                        1
protocolIdentifier                      6
sourceIPv4Address              27.3.0.196
sourceTransportPort                 59145
octetDeltaCount_Rev                   NaN
packetDeltaCount_Rev                  NaN
Name: 570, dtype: object

In [ ]:
similar_data_indices = []
iter = 0
for i in score_list:
  # print(i)
  if i<=0.19372687:
    similar_data_indices.append(iter)
  iter+=1

In [ ]:
similar_data_indices

[8,
 12,
 21,
 23,
 29,
 32,
 49,
 53,
 55,
 57,
 99,
 104,
 158,
 161,
 200,
 203,
 241,
 244,
 268,
 271,
 288,
 291,
 301,
 304,
 404,
 408,
 452,
 455,
 570]

In [ ]:
for i in similar_data_indices:
  print(df.iloc[i])

biFlowEndMilliseconds       1553070776019
biFlowStartMilliseconds     1553070776019
destinationIPv4Address         9.66.11.14
destinationTransportPort               22
octetDeltaCount                        44
packetDeltaCount                        1
protocolIdentifier                      6
sourceIPv4Address              27.3.0.196
sourceTransportPort                 59145
octetDeltaCount_Rev                   NaN
packetDeltaCount_Rev                  NaN
Name: 8, dtype: object
biFlowEndMilliseconds       1553070777019
biFlowStartMilliseconds     1553070777019
destinationIPv4Address         9.66.11.14
destinationTransportPort               22
octetDeltaCount                        44
packetDeltaCount                        1
protocolIdentifier                      6
sourceIPv4Address              27.3.0.196
sourceTransportPort                 59146
octetDeltaCount_Rev                   NaN
packetDeltaCount_Rev                  NaN
Name: 12, dtype: object
biFlowEndMilliseconds       1